# DMS Model v2 Demo. 

Run the following cell in order to apply the necessary imports and to be able to use functions used by multiple cells.

In [ ]:
""" Shared code snippets """

### Imports

from ultralytics import YOLO
import cv2
from PIL import Image
from matplotlib import pyplot as plt

###

### Constants
YOLOV8n_MODEL_PATH = 'models/yolov8n.pt'
YOLOV8n_MODEL_PATH = 'runs/detect/train5/weights/best.pt'
EYES_ONLY_MODEL_PATH = 'models/eyes.pt'
DMS_V2_MODEL_PATH = 'models/dms_v2.pt'

IMAGES_PATH = 'sample_tests/images'
IMAGES_RESULTS_PATH = 'sample_tests/images/results'

VIDEOS_PATH = 'sample_tests/videos'
VIDEOS_RESULTS_PATH = 'sample_tests/videos/results'


###

### Functions

# Resize camera/video frame to have consistent dimensions for all displays
def resize_frame(frame):
    # Define the desired width and height for displaying
    desired_width = 800
    desired_height = 600

    # Get the original width and height of the image
    original_height, original_width, _ = frame.shape

    # Calculate the aspect ratio of the original image
    aspect_ratio = original_width / original_height

    # Calculate the new dimensions while preserving the aspect ratio
    if aspect_ratio > 1:
        # Landscape orientation
        new_width = desired_width
        new_height = int(new_width / aspect_ratio)
    else:
        # Portrait orientation or square
        new_height = desired_height
        new_width = int(new_height * aspect_ratio)

    # Resize the image to the new dimensions
    resized_frame = cv2.resize(frame, (new_width, new_height))

    return resized_frame


def video_feed_and_results(model, video_source):
    # Open the video file or one of the available camera devices
    cap = video_source  # Change the camera index if needed

    # Loop through the video frames
    while True:
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLOv8 inference on the frame
            results = model(frame)

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            resized_image = resize_frame(annotated_frame)

            # Display the annotated frame
            cv2.imshow("DMS Model V2 Inference", resized_image)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break

    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()

###

### First, let's try the original yolov8n model, which can detect 80 different classes. 

In [ ]:
# Import the pretrained YOLOv8n model
model = YOLO(YOLOV8n_MODEL_PATH)

# Display model information
model.info()

# Get the list of object names (objects that the model is able to detect)
object_names = model.names

# Print the list of object names
print(object_names)

# Open the video file or one of the available camera devices
cap = cv2.VideoCapture(1)  # Change the camera index if needed or specify video path

video_feed_and_results(model=model, video_source=cap)

In [ ]:
!yolo val model=runs/detect/train5/weights/best.pt data=/kaggle/input/final/data.yaml split=val

## Second, let's test the custom YOLOv8 model (dms-model-v2).
#### The model can detect the following classes:
- Open Eye
- Closed Eye
- Cigarette
- Phone
- Seatbelt

In [ ]:
# Load Custom YOLOv8 Model
model = YOLO(DMS_V2_MODEL_PATH)

# Display model information
model.info()

# Get the list of object names (objects that the model is able to detect)
object_names = model.names

# Print the list of object names
print(object_names)

# Open the video file or one of the available camera devices
cap = cv2.VideoCapture(1)  # Change the camera index if needed 

### Or specify a video path
# video_file_name = 'v4.mp4'
# cap = cv2.VideoCapture(f'{VIDEOS_PATH}/{video_file_name}')  # Change the camera index if needed or specify video path
###

video_feed_and_results(model=model, video_source=cap)

To save video results:
(results will be saved at ./runs/detect/predict#/filename)

In [ ]:
# Load Custom YOLOv8 Model
model = YOLO(YOLOV8n_MODEL_PATH)
model.predict(source=f'{VIDEOS_PATH}/v5.mp4', save=True, conf=0.4, show=True)

It's also possible to use a YOLO command to run inference on a video or camera input and save the results to a file.

In [ ]:
!yolo predict model=models/dms_v2.pt source=1 show=True

## Third, testing the model on images and showing the results.

In [ ]:
# Load Custom YOLOv8 Model
model = YOLO(DMS_V2_MODEL_PATH)

img_file_name = '10.jpg'

# Load the input image
img = cv2.imread(f'{IMAGES_PATH}/{img_file_name}')

# Run inference on an image
results = model(img)  # results list

# Visualize the results on the frame
annotated_frame = results[0].plot()

#Show the image with matplotlib
plt.imshow(annotated_frame)
plt.show()

# View results
for r in results:
    print(r.boxes.data)  # print the Boxes object containing the detection bounding boxes

# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    im.save(f'{IMAGES_RESULTS_PATH}/results - {img_file_name}')  # save image